# ShowCase Exapmles
this is a file that contains codes that were removed but are importat, since they were used to test some theories

In [1]:
from utils.utils import *
from utils.ee_utils import *
from utils.utils import TextColors as tc
import ee
import geemap
from rois import ROIS
rois = ROIS()

In [2]:
ee.Initialize()
Map = geemap.Map()

In [3]:
vis_params_s1_bright = {
    'min': -10,
    'max': 10,
    'bands': ['VV_db']
}

## High cloud cover results in false snow_cover result in sentinel 2 images

For exapmle in this case, we are testing an image in summer in paris, but as you can see we are seeing a high snow/ice coverege and that is because there is a high snow covege.

In [4]:
roi = rois.bercy

s2_test_col = ee.ImageCollection('COPERNICUS/S2_SR') \
                    .filterDate('2021-05-17', '2021-05-19') \
                    .filterBounds(roi)
                    
sen2_print(s2_test_col)
s2_test_img = s2_test_col.first()

scl = s2_test_img.select('SCL')
mask = scl.eq(11)

Map = geemap.Map()
Map.addLayer(s2_test_img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 Image', True)
Map.addLayer(mask.updateMask(mask), {'palette': 'blue'}, 'Water Mask')
Map.centerObject(roi, zoom=10)
Map.addLayerControl()
Map


Map(center=[48.843641547684896, 2.380256652833762], controls=(WidgetControl(options=['position', 'transparent_…

## Sentinel 1A and 1B, Tandem Orbit 
sentinel 1a and 1b orbits result in a pretty simmilar, becuse of the short Spatial BaseLine.

- Although I still don't know, how can you determine whether two, realtive orbit numbers are tandem or not. -> <span style="color: #FF1111;">More Research Needed</span>

<span style="color: #11FF11;">Found The Answer - Read The Cells Below</span>


In [5]:
# Overwrite the default print function to for a consicise print
sen1_print = lambda s2_collection: ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                                    'relativeOrbitNumber_start', 'orbitProperties_pass',
                                                                                                    'cycleNumber'], first_is_t_in_millis=True)

In [6]:
roi = rois.paris
s2_col,s1_col = s1s2(roi,date = ('2021-07-15', '2021-08-15'),priority_path = 'DESCENDING',max_cloud = 100,max_snow = 100,month_span = 1,retry_days=0)

◍◍Finding S2
◍Single scene coverege was fount!
◍◍Finding S1
mean date:  2021-07-21
final date range:  2021-06-21 2021-08-21
Snowy days       :  []
Snowy days Buffed:  []
◍checking for DESCENDING single scene
Mosiac Covers 93.96914404374861 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted
collection size before removing snowy dates:  20
collection size after removing snowy dates:  20
◍Collection Found!


There is a pattern `BA AB BA AB BA...` with each `BA` differing only **1** day and each `BA AB` having a **5** day difference

In [7]:
sen1_print(s1_col.sort('system:time_start'))

,Property,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,system:time_start,2021-06-25,2021-06-26,2021-07-01,2021-07-02,2021-07-07,2021-07-08,2021-07-13,2021-07-14,2021-07-19,2021-07-20,2021-07-25,2021-07-26,2021-07-31,2021-08-01,2021-08-06,2021-08-07,2021-08-12,2021-08-13,2021-08-18,2021-08-19
1,platform_number,B,A,A,B,B,A,A,B,B,A,A,B,B,A,A,B,B,A,A,B
2,relativeOrbitNumber_start,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,164,234,235,164,165,235,236,165,166,236,237,166,167,237,238,167,168,238,239,168


<span style="color: #dc0fff;">Let's devide this imageCollection, into two collections of `Only 1A` and `Only 1B`</span>

Hmmm... as you can see, the `1A` collection has `two` diffrent `relativeOrbitnumber`s which mean there are two diffrent scences covering this roi, form two diffrent angles!






In [8]:
sen1_a =s1_col.filter(ee.Filter.eq('platform_number', 'A')).sort('system:time_start')
sen1_print(sen1_a)

,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-26,2021-07-01,2021-07-08,2021-07-13,2021-07-20,2021-07-25,2021-08-01,2021-08-06,2021-08-13,2021-08-18
1,platform_number,A,A,A,A,A,A,A,A,A,A
2,relativeOrbitNumber_start,110,8,110,8,110,8,110,8,110,8
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,234,235,235,236,236,237,237,238,238,239


Let's visualize this:

In [10]:
sen1a_110 = toDb(sen1_a.filter(ee.Filter.eq('relativeOrbitNumber_start', 110)).sort('system:time_start').map(toLinear).mean().clip(roi))
sen1a_8 = toDb(sen1_a.filter(ee.Filter.eq('relativeOrbitNumber_start', 8)).sort('system:time_start').map(toLinear).mean().clip(roi))
left_layer = geemap.ee_tile_layer(sen1a_110, vis_params_s1_bright, 'sen1a_110')
right_layer = geemap.ee_tile_layer(sen1a_8, vis_params_s1_bright, 'sen1a_8')
Map = geemap.Map(zoom=12)
Map.centerObject(roi)
Map.split_map(left_layer, right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

The same for 1B

In [ ]:
sen1_print(s1_col.filter(ee.Filter.eq('platform_number', 'B')).sort('system:time_start'))

,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-25,2021-07-02,2021-07-07,2021-07-14,2021-07-19,2021-07-26,2021-07-31,2021-08-07,2021-08-12,2021-08-19
1,platform_number,B,B,B,B,B,B,B,B,B,B
2,relativeOrbitNumber_start,8,110,8,110,8,110,8,110,8,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,164,164,165,165,166,166,167,167,168,168


### Let's deivde them by Oribt Number
`1A` and `1B` have the same Orbit Number over an Area!
- This is the Tandem Orbit
- So for consistat viewing geometry, filter by orbit number

In [12]:
print(tc.BOLD,'Relative orbit number 8: ',tc.ENDC)
sen1_print(s1_col.filter(ee.Filter.eq('relativeOrbitNumber_start', ee.Number(8))).sort('system:time_start'))


 Relative orbit number 8:  


,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-25,2021-07-01,2021-07-07,2021-07-13,2021-07-19,2021-07-25,2021-07-31,2021-08-06,2021-08-12,2021-08-18
1,platform_number,B,A,B,A,B,A,B,A,B,A
2,relativeOrbitNumber_start,8,8,8,8,8,8,8,8,8,8
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,164,235,165,236,166,237,167,238,168,239


In [13]:
print(tc.BOLD,'Relative orbit number 110: ',tc.ENDC)
sen1_print(s1_col.filter(ee.Filter.eq('relativeOrbitNumber_start', ee.Number(110))).sort('system:time_start'))

 Relative orbit number 110:  


,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-26,2021-07-02,2021-07-08,2021-07-14,2021-07-20,2021-07-26,2021-08-01,2021-08-07,2021-08-13,2021-08-19
1,platform_number,A,B,A,B,A,B,A,B,A,B
2,relativeOrbitNumber_start,110,110,110,110,110,110,110,110,110,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,234,164,235,165,236,166,237,167,238,168


### Comparing 1A and 1B on the same Orbit
lest filter the collection by only `orbit==8`, and then devide it into `1A` and `1B`

In [14]:
s1_col_8 = s1_col.filter(ee.Filter.eq('relativeOrbitNumber_start', ee.Number(8))).sort('system:time_start').map(toLinear)

In [15]:
s1a = toDb(s1_col_8.filter(ee.Filter.eq('platform_number', 'A')).mean().clip(roi))
s1b = toDb(s1_col_8.filter(ee.Filter.eq('platform_number', 'B')).mean().clip(roi))
left_layer = geemap.ee_tile_layer(s1a, vis_params_s1_bright, 'S1a')
right_layer = geemap.ee_tile_layer(s1b, vis_params_s1_bright, 'S1b')

<span style="color: #11FF11;">The Diffrence is a lot Less!</span>

- Corner reflectors stay a corner reflector! unlike diffrent orbits, these are `directional corner reflectors` the since the the dirrection in orbits change they might be a CR in one of them and not in the other!

- The spacle is higher since we devided the collection both by `A` and `B` and `relativeOrbit`

But we made out point!

Which  is: <span style="font-weight: bold; color: red;">USE ONLY ONE ORBIT</span>.

In [17]:
Map = geemap.Map(zoom=12)
Map.centerObject(roi)
Map.split_map(left_layer, right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…